In [3]:
import os
import pathlib

inputpath = '/home/nps2dsoccer/logs/archive.robocup.info/Soccer/Simulation/2D/logs/'
outputpath = '/home/nps2dsoccer/logs/dataset/shoots/'

count = 0
for log_file in pathlib.Path(inputpath).glob('**/*.rcg.gz'):
    count = count + 1
    if count < 4 :
            
        file_length = len(log_file.stem)
        print(1, log_file.stem, file_length)
        com_filepath = str(log_file)
        # toll = len(com_filepath)
        print(2, com_filepath[len(inputpath): len(com_filepath) - file_length - 3])
        structure = os.path.join(outputpath, com_filepath[len(inputpath): len(com_filepath) - file_length - 3 ]) #  3 is used to adjust the '.gz' of the file name that is not considered by stem
        print(3, structure)

        if not os.path.isdir(structure):
            os.makedirs(structure)
        gen_shoot_situations(log_file, structure)


1 20190704094758-Fractals2019_2-vs-MT2019_1.rcg 45
2 RoboCup/2019/SeedingRound/GroupA/
3 /home/nps2dsoccer/logs/dataset/shoots/RoboCup/2019/SeedingRound/GroupA/
(show 888 ((b) 47.7788 5.7642 -0.5 -2.082) ((l 1) 7 0x9 -50.8907 1.8621 0 -0 90.825 -90 (v h 180) (s 8000 0.950193 1 128245) (f l 9) (c 0 46 1055 0 1 1102 7 45 0 0 683)) ((l 2) 0 0x1 0.0147 -0.2565 0.0953 0.0243 -59.577 35 (v h 180) (s 7946 1 1 103958) (f l 9) (c 0 279 812 0 1 1092 66 163 0 0 315)) ((l 3) 16 0x1 -3.5509 13.399 0.0006 0 -54.079 8 (v h 180) (s 8000 0.977032 1 102748) (f l 9) (c 10 284 792 0 1 1087 53 189 0 0 331)) ((l 4) 17 0x40001 8.0475 -11.9788 0.0074 0.0042 -107.78 90 (v h 120) (s 7148.87 0.970289 1 103448) (f l 9) (c 0 292 788 0 1 1082 69 141 1 0 298)) ((l 5) 5 0x1 0.835 23.2975 0.1965 -0.0315 -9.183 -72 (v h 180) (s 7833.72 0.98095 1 100418) (f l 9) (c 2 322 752 0 1 1077 58 102 0 0 470)) ((l 6) 13 0x1 36.1893 7.7196 0.2264 0.0535 13.885 -44 (v h 60) (s 5036.68 0.913251 1 91825.7) (f l 9) (c 19 462 590 0 1 1

In [2]:
from collections import deque

def gen_shoot_situations(src, dest): 
    line_history = deque(maxlen=15)  # Decided based on matched of 2019 Elimination Round (Max 15 cycle)
    cycle_history = deque(maxlen=3)  # Safely allocated to read a situation after discussion with Thiago
    potent_cycle = deque(maxlen=3)
    with open(src) as inputs:
        for line in inputs:
            if " goal_l" in line : 
                
                cycle_slice = find_shoot(line_history, cycle_history, potent_cycle )
                print(cycle_slice)
                
                # print(*line_history, sep='\n')

                # for cycle in line_history:
                #     if " 0x3 " in cycle  : # check if kick is found any cycle prior to the goal 
                #         cycle_history.append(cycle)
                #         potent_cycle = cycle_history.copy()
                        
                #     else:
                #         # to keep just the two cycle prior where the kick is found
                #         cycle_history.append(cycle)
                
                # # print(cycle_history.popleft())  # pop the last two clcyle prior where the kick is found
                # cycle_slice = potent_cycle.popleft()
                # print(cycle_slice)
                print( line, 'L')

                # gen_json(count, slice , dest)

                
                # Clear history so if two goal_l/goal_r seen in close proximity, we don't
                # echo some lines twice
                # line_history.clear()
            elif " goal_r" in line:

                cycle_slice = find_shoot(line_history, cycle_history, potent_cycle )
                print(cycle_slice)
                print( line, 'R')

            else:
                # When deque reaches 15 lines, will automatically evict oldest
                line_history.append(line)
                

In [1]:
def find_shoot(line_history, cycle_history, potent_cycle):
    for cycle in line_history:
        if " 0x3 " in cycle  : # check if kick is found any cycle prior to the goal 
            cycle_history.append(cycle)
            potent_cycle = cycle_history.copy()
            
        else:
            # to keep just the two cycle prior where the kick is found
            cycle_history.append(cycle)

    # print(cycle_history.popleft())  
    cycle_slice = potent_cycle.popleft() # pop the two clcyle prior where the kick is found (two cycle a buffer)
    # print(cycle_slice)
    line_history.clear()
    return cycle_slice


In [ ]:
def gen_json(count, slice, dest, side):
    